# JRC2018 ROI

This notebook shows how to download neuropile segments (ROIs) for JRC2018U brain template. The ROIs are stored in virtual fly brain, in the dataset with id as JRC2018. 

Templates are stored as nrrd files, with a resolutoin of (1210,566,174)(x:0.52,y:0.52,z:0.1 microns). 

All ROIs are saved in lsa-mcdb-jclowney/lab/Computational_tools/CMTK/templates/JRC2018_ROIs, including nrrd files and a .csv table for their identity.

Reference: [Bogovic et al., 2019](https://v2.virtualflybrain.org/org.geppetto.frontend/org.geppetto.frontend?id=doi_10_1101_376384)

In [6]:
import os 
import pathlib
# connect to virtual fly brain
from vfb_connect import vfb

# set working directory to store ROIs on the server
work_dir=pathlib.Path("/Volumes/lsa-mcdb-jclowney/lab/Computational_tools/CMTK/templates/JRC2018_ROIs")


# set up download function
def download_nrrd_image(vc, vfb_id, output_folder, template="JRC2018U"):
    """Downloads a single NRRD image from VFB for a given ID.

    This function retrieves information for a given Virtual Fly Brain (VFB) ID,
    finds the image URL for a specified template, and downloads it as an
    NRRD file. The output folder will be created if it doesn't exist.

    Args:
        vc: An instance of vfb_connect.cross_server_tools.VfbConnect.
        vfb_id (str): The VFB ID of the image to download (e.g., "VFB_00102745").
        output_folder (str): The path to the folder where the image will be saved.
        template (str, optional): The template space of the image to download.
            Defaults to "JRC2018U".
    """
    import os
    import urllib.request
    import re
    
    try:
        vcinfo = vc.neo_query_wrapper.get_anatomical_individual_TermInfo([vfb_id], summary=False)
        if not vcinfo:
            print(f"No information found for VFB ID: {vfb_id}")
            return
    except Exception as e:
        print(f"Failed to retrieve information for {vfb_id}. Error: {e}")
        return

    term_info = vcinfo[0].get('term', {})
    label = term_info.get('core', {}).get('label', 'unknown_label')
    label = label.split(" ")[0]
    short_form = term_info.get('core', {}).get('short_form', vfb_id)

    # Sanitize label and short_form for use in a filename
    safe_label = re.sub(r'[\\/*?:"<>|]', "", label)
    safe_short_form = re.sub(r'[\\/*?:"<>|]', "", short_form)

    os.makedirs(output_folder, exist_ok=True)

    found_image = False
    for i, ch in enumerate(vcinfo[0].get('channel_image', [])):
        try:
            template_symbol = ch['image']['template_anatomy']['symbol']
            print(f"Channel {i}: Found image with template '{template_symbol}'.")
            if template_symbol == template:
                image_link = ch['image']['image_nrrd']
                
                file_name = f"{safe_label}_{safe_short_form}_{template_symbol}.nrrd"
                file_path = os.path.join(output_folder, file_name)
                
                print(f"Downloading {file_name} to {output_folder}...")
                urllib.request.urlretrieve(image_link, file_path)
                print("Download complete.")
                found_image = True
        except KeyError:
            print(f"Channel {i}: No template information or NRRD link found.")
        except Exception as e:
            print(f"An error occurred while processing channel {i}: {e}")

    if not found_image:
        print(f"No image found with the template '{template}' for VFB ID {vfb_id}.")


In [3]:
# Step 1: download the ROI list 
JRC2018 = vfb.get_instances_by_dataset('JRC2018')
JRC2018.to_csv(work_dir / "JRC2018_ROI_list.csv")
JRC2018.head()

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,xrefs,templates,dataset,license,description
0,SIP on JRC2018Unisex adult brain,,VFB_00102164,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[superior intermediate protocerebrum],[FBbt_00045032],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
1,ATL on JRC2018Unisex adult brain,,VFB_00102190,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[antler],[FBbt_00045039],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
2,SLP on JRC2018Unisex adult brain,,VFB_00102162,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[superior lateral protocerebrum],[FBbt_00007054],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
3,IB on JRC2018Unisex adult brain,,VFB_00102185,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[inferior bridge],[FBbt_00040050],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
4,LAL on JRC2018Unisex adult brain,,VFB_00102140,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[adult lateral accessory lobe],[FBbt_00003681],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN


In [4]:
# establish connection
from vfb_connect.cross_server_tools import VfbConnect
vc = VfbConnect()

Welcome to the Virtual Fly Brain API
See the documentation at: https://virtualflybrain.org/docs/tutorials/apis/

Establishing connections to https://VirtualFlyBrain.org services...
Session Established!

Session Established!



In [8]:
# download all ROIs
for i, vfb_id in enumerate(JRC2018['id'].tolist()):
    download_nrrd_image(vc, vfb_id, work_dir)

print(f"Dolwnloaded {i} ROIs.")

Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.

## Flybrain Mask Generation

The following code generates two important NRRD mask files and a lookup table from the downloaded ROIs.

### 1. Central Brain Binary Mask (`JRC2018U_central_brain_mask.nrrd`)
This is a simple binary mask of the central brain neuropile.
- **Voxels inside** any central brain neuropile are assigned a value of **255**.
- **Voxels outside** the central brain are **0**.
This file is useful when you need to isolate the entire central brain region from the background.

### 2. Central Brain Label Mask (`JRC2018U_central_brain_ROIs.nrrd`)
This is a single image where each central brain neuropile is represented by a unique integer ID.
- Each voxel's value corresponds to the ID of the neuropile it belongs to.
- If neuropiles overlap, the voxel will be assigned the ID of the last neuropile processed in the loop.

### 3. ROI Lookup Table (`JRC2018U_central_brain_ROI_lookup_table.csv`)
This CSV file is the key to understanding the `ROIs.nrrd` label mask. It connects the integer IDs in the mask back to their biological names.

**How to use it:**
1. Open the `lookup_table.csv` file.
2. Find the neuropile you are interested in (e.g., "ellipsoid body").
3. Note its corresponding `ROI_number`.
4. In the `JRC2018U_central_brain_ROIs.nrrd` image, all voxels with that number belong to the ellipsoid body.

In [ ]:
import os 
import pathlib
import pandas as pd
import numpy as np
import nrrd

# set working directory to store ROIs on the server
work_dir=pathlib.Path("/Volumes/lsa-mcdb-jclowney/lab/Computational_tools/CMTK/templates/JRC2018_ROIs")

# read the ROI table
roi_table=pd.read_csv(work_dir / "JRC2018_neuropiles.csv", index_col=0)
# reformate the table
roi_table["neuropile_name"] = roi_table["parents_label"].str.strip("['']")
roi_table.head()

,Unnamed: 0,label,path,region,symbol,id,tags,parents_label,parents_id,data_source,accession,xrefs,templates,dataset,license,description,neuropile_name
0,0,SIP on JRC2018Unisex adult brain,SIP_VFB_00102164_JRC2018U.nrrd,central,NaN,VFB_00102164,"['Entity', 'Individual', 'VFB', 'Adult', 'Anat...",['superior intermediate protocerebrum'],['FBbt_00045032'],[],[],[],['JRC2018Unisex'],['JRC2018'],['https://creativecommons.org/licenses/by-nc-s...,NaN,superior intermediate protocerebrum
1,1,ATL on JRC2018Unisex adult brain,ATL_VFB_00102190_JRC2018U.nrrd,central,NaN,VFB_00102190,"['Entity', 'Individual', 'VFB', 'Adult', 'Anat...",['antler'],['FBbt_00045039'],[],[],[],['JRC2018Unisex'],['JRC2018'],['https://creativecommons.org/licenses/by-nc-s...,NaN,antler
2,2,SLP on JRC2018Unisex adult brain,SLP_VFB_00102162_JRC2018U.nrrd,central,NaN,VFB_00102162,"['Entity', 'Individual', 'VFB', 'Adult', 'Anat...",['superior lateral protocerebrum'],['FBbt_00007054'],[],[],[],['JRC2018Unisex'],['JRC2018'],['https://creativecommons.org/licenses/by-nc-s...,NaN,superior lateral protocerebrum
3,3,IB on JRC2018Unisex adult brain,IB_VFB_00102185_JRC2018U.nrrd,central,NaN,VFB_00102185,"['Entity', 'Individual', 'VFB', 'Adult', 'Anat...",['inferior bridge'],['FBbt_00040050'],[],[],[],['JRC2018Unisex'],['JRC2018'],['https://creativecommons.org/licenses/by-nc-s...,NaN,inferior bridge
4,4,LAL on JRC2018Unisex adult brain,LAL_VFB_00102140_JRC2018U.nrrd,central,NaN,VFB_00102140,"['Entity', 'Individual', 'VFB', 'Adult', 'Anat...",['adult lateral accessory lobe'],['FBbt_00003681'],[],[],[],['JRC2018Unisex'],['JRC2018'],['https://creativecommons.org/licenses/by-nc-s...,NaN,adult lateral accessory lobe


In [12]:
# get the list of central brain neuropiles
central_brain_files = roi_table.loc[roi_table["region"] == "central", ["path"]].values.tolist()

ROI_mask = None
central_brain_mask = None
ROI_numbers = len(central_brain_files)

for i, file in enumerate(central_brain_files):
    # read the nrrd file
    data, header = nrrd.read(work_dir / file[0])
    if ROI_mask is None:
        ROI_mask = np.zeros(data.shape, dtype=np.uint8)
        central_brain_mask = np.zeros(data.shape, dtype=np.uint8)
    # assign a unique integer value to each neuropile
    ROI_mask[data > 0] = i + 1  # start from 1
    central_brain_mask[data > 0] = 255  # binary mask

# save the masks
nrrd.write(str(work_dir / "JRC2018U_central_brain_ROIs.nrrd"), ROI_mask, header)
nrrd.write(str(work_dir / "JRC2018U_central_brain_mask.nrrd"), central_brain_mask, header)

# create a lookup table
lookup_table = roi_table[roi_table["region"] == "central"][["neuropile_name"]].copy()
lookup_table["ROI_number"] = np.arange(1, ROI_numbers + 1)
lookup_table.to_csv(str(work_dir / "JRC2018U_central_brain_ROI_lookup_table.csv"))